In [ ]:
import numpy as np
import torch

from preprocessing import get_tracking_and_plays, compute_feature_df, create_feature_tensor
from model import PREPROCESS_STD, PREPROCESS_MEAN, load_expected_yards_model
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import pandas as pd
import os

In [35]:
week1 = pd.read_csv("CounterFactualsweek_1.csv")
print(week1.columns)
print(sum(week1['Prediction Difference']))

Index(['Unnamed: 0', 'nflId', 'gameId', 'playId', 'frameId', 'Yards Remaining',
       'Original Prediction', 'Projected Prediction', 'Prediction Difference'],
      dtype='object')
11765.29178738594


In [ ]:
print(cur_x[1])

In [ ]:
def get_tracking_with_plays(filepath):
    tracking = pd.read_csv(filepath)
    plays = pd.read_csv("plays.csv")
    tracking = tracking[tracking['playId'].isin(tracking[tracking['event'] != 'fumble']['playId'].unique())]
    plays = plays[plays['playNullifiedByPenalty'] == 'N']
    tracking.loc[tracking['playDirection'] == 'left', 'x'] = 120 - tracking.loc[tracking['playDirection'] == 'left', 'x']
    tracking.loc[tracking['playDirection'] == 'left', 'y'] = (160/3) - tracking.loc[tracking['playDirection'] == 'left', 'y']
    tracking.loc[tracking['playDirection'] == 'left', 'dir'] += 180
    tracking.loc[tracking['dir'] > 360, 'dir'] -= 360
    tracking.loc[tracking['playDirection'] == 'left', 'o'] += 180
    tracking.loc[tracking['o'] > 360, 'o'] -= 360
    tracking_with_plays = tracking.merge(plays, on=['gameId', 'playId'], how='left')
    tracking_with_plays['is_on_offense'] = tracking_with_plays['club'] == tracking_with_plays['possessionTeam']
    tracking_with_plays['is_on_defense'] = tracking_with_plays['club'] == tracking_with_plays['defensiveTeam']
    tracking_with_plays['is_ballcarrier'] = tracking_with_plays['ballCarrierId'] == tracking_with_plays['nflId']
    bc_coords=tracking_with_plays.loc[tracking_with_plays['is_ballcarrier']]
    bc_coords['bc_x']=bc_coords['x']
    bc_coords['bc_y']=bc_coords['y']
    bc_coords=bc_coords[['gameId', 'playId', 'frameId', 'bc_x', 'bc_y']]
    tracking_with_plays=tracking_with_plays.merge(bc_coords, on=['gameId', 'playId', 'frameId'], how='left')
    end_frame = tracking_with_plays[tracking_with_plays['event'].isin(['tackle', 'out_of_bounds'])].groupby(['gameId', 'playId'])['frameId'].min().reset_index()
    end_frame.rename(columns={'frameId': 'frameId_end'}, inplace=True)
    start_frame = tracking_with_plays[tracking_with_plays['event'].isin(['run', 'lateral', 'run_pass_option', 'handoff', 'pass_arrived'])].groupby(['gameId', 'playId'])['frameId'].min().reset_index()
    start_frame.rename(columns={'frameId': 'frameId_start'}, inplace=True)
    tracking_with_plays = tracking_with_plays.merge(start_frame, on=['gameId', 'playId'], how='left')
    tracking_with_plays = tracking_with_plays.merge(end_frame, on=['gameId', 'playId'], how='left')
    tracking_with_plays = tracking_with_plays[(tracking_with_plays['frameId'] <= tracking_with_plays['frameId_end']) &
                                              (tracking_with_plays['frameId'] >= tracking_with_plays['frameId_start'])]
    return tracking_with_plays

In [ ]:
print(len(tracking1['frameId'])//22,cur_x.shape)

In [ ]:
def create_input_tensors(files):
    i = 1
    for c in files:
        print('Working on file ' + c)
        tracking_with_play_train = get_tracking_and_plays(c)
        feature_df_train = compute_feature_df(tracking_with_play_train)
        input_tensor, target_tensor, reference_tensor = create_feature_tensor(feature_df_train)
        np.save(f'week_{i}_full_x.npy', input_tensor)
        np.save(f'week_{i}_full_y.npy', target_tensor)
        np.save(f'week_{i}_full_r.npy', reference_tensor)


In [ ]:
!nvidia-smi

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
create_input_tensors([os.path.join(os.path.join("Data", str(2024)),"tracking_week_"+str(1) +".csv")])

In [ ]:
cur_r = np.load(f'week_{1}_r_new.npy')
print(len(np.unique(cur_r['gameId'])),len(cur_r['frameId']))

In [ ]:
test_x_arrays = []
test_y_arrays = []
test_r_arrays = []
for i in range(1,10):
    #print(i)
    cur_x = np.load(f'week_{i}_x_new.npy')
    cur_y = np.load(f'week_{i}_y_new.npy')
    cur_r = np.load(f'week_{i}_r_new.npy')
    test_x_arrays.append(cur_x)
    test_y_arrays.append(cur_y)
    test_r_arrays.append(cur_r)
    print(np.unique(cur_r[:,0]))

test_x_tensor = np.concatenate(test_x_arrays)
test_y_tensor = np.concatenate(test_y_arrays)
test_r_tensor = np.concatenate(test_r_arrays)
print(len(np.unique(test_r_tensor[:,0])))

In [ ]:
test_x = torch.tensor(test_x_tensor, dtype=torch.float)
test_y = torch.tensor(test_y_tensor, dtype=torch.float)
test_r = torch.tensor(test_r_tensor, dtype=torch.int)

# Normalize data according to training set mean
test_x = (test_x - PREPROCESS_MEAN) / PREPROCESS_STD

In [ ]:
print(len(np.unique(test_r[:,0])))
print(len(np.unique(test_r_tensor[:,0])))

In [ ]:
test_dataset = TensorDataset(test_x, test_y, test_r)

test_loader = DataLoader(test_dataset)

In [ ]:
model = load_expected_yards_model()
model.eval()

all_outputs = np.zeros((len(test_x), 5))
cur = 0
for cur_x, cur_y, cur_r in tqdm(test_loader):
    cur_outputs = model(cur_x)
    for i in range(len(cur_outputs)):
        all_outputs[cur, 0] = cur_r[i][0]
        all_outputs[cur, 1] = cur_r[i][1]
        all_outputs[cur, 2] = cur_r[i][2]
        all_outputs[cur, 3] = cur_outputs[i]
        all_outputs[cur, 4] = cur_y[i]
        cur+=1

In [ ]:
all_outputs[-1]
#sum(all_outputs[])

In [ ]:
df = pd.DataFrame(all_outputs, columns=['gameId', 'playId', 'frameId', 'ExpectedYards', 'ActualYards'])

In [ ]:
df.info()

In [ ]:
df.to_csv('ExpectedYards_v1.csv')

In [ ]:
print(sum(df['ExpectedYards']),sum(df['ActualYards']))